In [223]:
1+1

2

In [1]:
import stanza, re, benepar, psutil, gc, json, csv, time, string

import numpy as np
import networkx as nx
import pandas as pd
import seaborn as sns

from textstat import textstat
from datetime import datetime
from collections import Counter

import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk import Tree
from textblob import Word
from spellchecker import SpellChecker

import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator, LogFormatter

In [50]:
def memcheck():
    gc.collect()
    memory_info = psutil.virtual_memory()

    # Display the memory information in GB
    total_memory = memory_info.total / (1024 ** 3)
    available_memory = memory_info.available / (1024 ** 3)
    used_memory = memory_info.used / (1024 ** 3)

    print(f"Total Memory: {total_memory:.2f} GB")
    print(f"Available Memory: {available_memory:.2f} GB")
    print(f"Used Memory: {used_memory:.2f} GB")

def time_taken(start_time, end_time):
    time_taken = end_time - start_time
    hours, remainder = divmod(time_taken.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"Time Taken: {hours} hours, {minutes} minutes, {seconds} seconds")

In [3]:
nlp_stanza = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse,sentiment,ner')
nlp_tokens = stanza.Pipeline(lang='en', processors='tokenize', use_gpu=False)

2024-09-12 18:12:22 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-09-12 18:12:22 INFO: Downloaded file to C:\Users\Roland\stanza_resources\resources.json
2024-09-12 18:12:23 INFO: Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| pos       | combined_charlm           |
| lemma     | combined_nocharlm         |
| depparse  | combined_charlm           |
| sentiment | sstplus_charlm            |
| ner       | ontonotes-ww-multi_charlm |

2024-09-12 18:12:23 INFO: Using device: cpu
2024-09-12 18:12:23 INFO: Loading: tokenize
2024-09-12 18:12:25 INFO: Loading: mwt
2024-09-12 18:12:25 INFO: Loading: pos
2024-09-12 18:12:26 INFO: Loading: lemma
2024-09-12 18:12:26 INFO: Loading: depparse
2024-09-12 18:12:26 INFO: Loading: sentiment
2024-09-12 18:12:27 INFO: Loading: ner
2024-09-12 18:12:27 INFO: Done loading processors!
2024-09-12 18:12:27 INFO: Checking for updates to resources.json in

2024-09-12 18:12:27 INFO: Downloaded file to C:\Users\Roland\stanza_resources\resources.json
2024-09-12 18:12:27 WARNING: Language en package default expects mwt, which has been added
2024-09-12 18:12:27 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| mwt       | combined |

2024-09-12 18:12:27 INFO: Using device: cpu
2024-09-12 18:12:27 INFO: Loading: tokenize
2024-09-12 18:12:27 INFO: Loading: mwt
2024-09-12 18:12:27 INFO: Done loading processors!


In [4]:
nlp_spacy = spacy.load('en_core_web_sm')

# Add the benepar parser to the pipeline

# tokenizer = T5Tokenizer.from_pretrained('t5-small', clean_up_tokenization_spaces=True)  
# default is currently True but will change to False

#nlp_spacy.add_pipe("benepar", config={"model": "benepar_en3"})

In [1]:
source_texts= list()
source_texts.append("Kazuo Ishiguro - Never Let Me Go.txt")
source_texts.append("Kazuo Ishiguro - The Remains of the Day")
source_texts.append("Kazuo Ishiguro - A Pale View of Hills-Knopf Doubleday Publishing Group (1990)")
source_texts.append("Kazuo-Ishiguro-When-We-Were-Orphans-Alfred-A.-Knopf_Vintage-_2001_")
source_texts.append("The Buried Giant (Kazuo Ishiguro) (Z-Library)-1")
source_texts.append("Kazuo Ishiguro - The Unconsoled-Vintage (1996)")
directory_path = "C:/Users/Roland/Documents/AI/stylometry/"
file_path = directory_path+source_texts[1]+".txt"
print (file_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day.txt


In [2]:
file_path = file_path[:-4]
print(file_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day


In [3]:
file_path = file_path + " - edited 1.txt"
print(file_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day - edited 1.txt


In [4]:
with open(file_path, 'r') as file:
    text = file.read()
print (len(text))

423412


In [9]:
save_path = file_path[:-5]
save_path = save_path + "2.txt"
print(save_path)

C:/Users/Roland/Documents/AI/stylometry/Kazuo Ishiguro - The Remains of the Day - edited 2.txt


In [ ]:
# process the text with stanza 

# Record the start time
start_time = datetime.now()
print(f"Starting processing at: {start_time.strftime('%H:%M:%S')}")

memcheck()
doc_stanza = nlp_stanza(text)
memcheck()

end_time = datetime.now()
print(f"Completed parsing at: {end_time.strftime('%H:%M:%S')}")

time_taken = end_time - start_time

# Print the time taken in hours, minutes, and seconds
hours, remainder = divmod(time_taken.seconds, 3600)
minutes, seconds = divmod(remainder, 60)
print(f"Time Taken: {hours} hours, {minutes} minutes, {seconds} seconds")

print(f"\ndoc_stanza has {len(doc_stanza.sentences)} sentences.")


In [10]:
def display_text_in_context(text, words, window = 75):
#Finds the first occurrence of each word in target text and displays it
    
    for word in words:
        ind = text.find(word)
        if ind == -1:
            print (f"{word} not found in text")
        else:
            display_text = text[ind-window:ind+window+1]
            display_text = display_text.replace("\n", " ")
            display_text = display_text.replace("\t", " ")
            print(display_text)

def display_words_in_context(text_words, words, repeat = False, window = 2):
# Finds the first occurrence of each word in target list of words. Optionally finds every subsequent occurrence     
    if not repeat:
        for word in words:
            try:
                index = text_words.index(word)
                #print(f"The index of '{word}' is: {ind}")
                display_text = ' '.join(text_words[index - window:index])
                display_text = display_text + " [" + word + "] "
                display_text = display_text + ' '.join(text_words[index + 1 : index + window + 1])
                print(display_text)
            except ValueError:
                print(f"'{word}' not found in the text.")
    else:
        for word in words:
            indices = [index for index, value in enumerate(text_words) if value == word]
            if len(indices) == 0:
                print(f"'{word}' not found in the text.")
            else:
                for index in indices:
                    display_text = ' '.join(text_words[index - window:index])
                    display_text = display_text + " [" + word + "] "
                    display_text = display_text + ' '.join(text_words[index + 1 : index + window + 1])
                    print(display_text)
                

In [53]:
def where_is(word):
    # finds if spell checker has flagged a word.  Must run spell checker first.
    if (word in spell_set_1): 
        print ("In spell set 1")
    if (word in blob_set_1):
        print ("In blob set 1")
    if (word in spell_set_1) or (word in blob_set_1):
        display_words_in_context(test_words_1, [word], False, 6)
    if (word in spell_set_2): 
        print ("In spell set 2")
    if (word in blob_set_2):
        print ("In blob  set 2")
    if (word in spell_set_2) or (word in blob_set_2):
        display_words_in_context(test_words_2, [word], False, 6)
    if (word in spell_set_3): 
        print ("In spell set 3")
    if (word in blob_set_3):
        print ("In blob set 3")
    if (word in spell_set_1) or (word in blob_set_3):
        display_words_in_context(test_words_3, [word], False, 6)        
        

In [5]:
def find_hyphenated_words(text):
    # Use regex to find words that are hyphenated across lines
    # Match sequences where a hyphen is at the end of a line, followed by a newline, and then continued with a word
    hyphenated_words = re.findall(r"(\w+)-\n(\w+)", text)

    for first_part, second_part in hyphenated_words:
        print(f"Found broken word: {first_part}-{second_part}")
    print(f"{len(hyphenated_words)} hyphenated words found, text length is: {len(text)}")
    


In [12]:
def replace_hyphenated_words(text, show_changes = False):
    # This regular expression captures words separated by a hyphen, with letters on both sides.
    pattern = r'(\b\w+)-(\w+\b)'
    
    # Function to handle replacement and printing
    def replacement(match):
        # Original hyphenated word
        original_word = match.group(0)
        # Replacement word (with space instead of hyphen)
        altered_word = match.group(1) + " " + match.group(2)
        
        # Print the hyphenated word that was altered
        if show_changes: print(f"Altered: {original_word} -> {altered_word}")
        
        return altered_word

    # Replace hyphenated words and call the replacement function
    result = re.sub(pattern, replacement, text)
    
    return result


Prepare words for spell checking.

The first set has punctuation removed

The second set has punctuation removed after replacing hyphenated words with spaces

The third set is also made lower case

In [13]:
test_words_1 = text.translate(str.maketrans('', '', string.punctuation)).split()
unhyphenated = replace_hyphenated_words(text)
test_words_2 = unhyphenated.translate(str.maketrans('', '', string.punctuation)).split()
test_words_3 = unhyphenated.translate(str.maketrans('', '', string.punctuation)).lower().split()

In [14]:
# Use spellchecker to validate words
# Initialize spell checker
spell = SpellChecker()


# Find words not in the dictionary
spell_set_1 = {word for word in test_words_1 if word not in spell}
spell_set_2 = {word for word in test_words_2 if word not in spell}
spell_set_3 = {word for word in test_words_3 if word not in spell}

print(len(spell_set_1), len(spell_set_2), len(spell_set_3))


500 414 394


In [15]:
print(f"errors found with punctuation removed only: {len(spell_set_1)}")
print(f"errors found with punctuation removed after replacing hyphens: {len(spell_set_2)}")
print(f"errors found with case lowered too:  {len(spell_set_3)}")     
print(f"errors eliminated by removing hyphens:  {len(spell_set_1 - spell_set_2)}")
print(f"errors introduced by removing hyphens:  {len(spell_set_2 - spell_set_1)}")
print(f"errors eliminated by lowering case:  {len(spell_set_2 - spell_set_3)}")
print(f"errors introduced by lowering case:  {len(spell_set_3 - spell_set_2)}")

errors found with punctuation removed only: 500
errors found with punctuation removed after replacing hyphens: 414
errors found with case lowered too:  394
errors eliminated by removing hyphens:  102
errors introduced by removing hyphens:  16
errors eliminated by lowering case:  246
errors introduced by lowering case:  226


In [16]:
oddities = list(spell_set_2 - spell_set_1)
print (' '.join(oddities))
print("\n")
# display_words_in_context(test_words_2, oddities, True, 3)

longue inwaiting Karl Heinz loverStevens pre ordinated sheetings martialling co ofhand empting Semitic vident humouredness un




Select the ones that could be errors and find them in the original text:

In [17]:
selected_oddities = ["team of ladies", "sleights-of-hand", "re a nature-lover", "have been self-vident"]
#These should appear in the comprehensive search later.

In [18]:
display_text_in_context(text, selected_oddities)

 been the great contrast in their backgrounds - bringing with them a large team of ladies-in-waiting and footmen, as well as a great many trunks. Then 
ts' designed to bring that extra bit out of staff, as well as the various 'sleights-of-hand' - the equivalent of a conjuror's - by which a butler could
ct of my arrival and held out his glass.  "I think it's admirable that you're a nature-lover,Stevens," he said, as I served him. "And I dare say it's a
ess of someone of your age and standing addressing him as 'William' should have been self-vident to you."  "Mr Stevens, I may not have been a housekeep


Select the errors and add them to the replacements dictionary.  The rest of the oddities are not spelling errors.

In [21]:
spell_set_2_lowered = {word.lower() for word in spell_set_2}
len(spell_set_2_lowered - spell_set_3)

0

Check words that are picked out by spell checker when made lower case:

In [22]:
case_oddities = list(spell_set_3 - spell_set_2)
print (' '.join(case_oddities))
print("\n")
# display_words_in_context(test_words_3, case_oddities, False, 3)

semitic granchester farradays britannica celts charleville reggie societys thornley eden yorkshire crosby saltash hardacres frau hertfordshire italian overath muggeridge versailles tavistock paris christmas hayes herr ritz simpson helston oswald chinaman american thatll taunton whittaker taylor attachã© loughborough compton belgians martha moscombe cornwall semitism mursden darlington dave james roosevelt charles symonss boer tj1e clementss ribbentrop ifi pennsylvania tory george bremann dr reginalds wakefields dorset blair lewiss britain churchill dupont ohnothing davidson frenchmen dickenson taylors catherines southampton mosleys imvery morgan karl ploughmans england carlisles catherine barnets jones saltman redding ruhr exeter morgans hitlers seamus mortimer bridewood eleanor rhineland wilkinson italians belgian englishman sunday ribbentrops mortimers giffen halifax bremanns bernard englishmen morphy europe wer dorothy harrys britons trevor silverss giffens nazis barnet johnson caro

Check words that picked out by spell checker with the original case but passed when made lower case:

In [23]:
inverse_case_oddities = list(spell_set_2 - spell_set_3)
print (' '.join(inverse_case_oddities))
print("\n")
# display_words_in_context(test_words_2, inverse_case_oddities, False, 3)

Davids Hitlers Hitler Victorian Isnt Hayes Eden Ritz Dorchester Allshot Thursday Reginalds Rayne Charleville Exeter Gentlemans Martha Granchester tJ1e Clementss Ohnothing Wakefields Boer Keynes Eleanor Infact Belgian Mustnt Muggeridge Frenchmen Bentley Duponts Versailles Mauvis Dignitys Jewish Ribbentrop Kenton KarlHeinz Astor Semitic Mr Mosleys Dorothy Dh Reggie Moscombe Barnet Giffens Helston Karl Mursden loverStevens Richard Mortimer James Taylors Salisbury Thatll Herman attachÃ© Belgians Winston Daniels Trevor Morgan September Englands Heinz Nazis Christmas Cornwall Carlisle Thornley Branbury Oswald OK Tory Ruhr Andrews Youre Farradayscircle Lloyd Charles Taunton Ive Darlington Dr Saltash Mortimers Couldnt William Wetherby Barnets Overath Americans Symons Laval Pennsylvania Semitism British Kentons Mrs Whittaker Frenchman Youd Alice Dickenson Britons Silverss Nellie Oxfordshire Seamus Chalmers Neighbourss Catherine Youve Dont London Churchill Leonard Benn Edward Chinaman Farraday S

Run text blob, an alternative spell checker:

In [51]:
start_time = datetime.now()
print(f"Starting processing at: {start_time.strftime('%H:%M:%S')}")



# Find invalid words
blob_set_1 = set()
for word in test_words_1:
    result = Word(word).spellcheck()
    if result[0][1] < 1.0 or result[0][0] != word:
        blob_set_1.add(word)

blob_set_2 = set()
for word in test_words_2:
    result = Word(word).spellcheck()
    if result[0][1] < 1.0 or result[0][0] != word:
        blob_set_2.add(word)

blob_set_3 = set()
for word in test_words_3:
    result = Word(word).spellcheck()
    if result[0][1] < 1.0 or result[0][0] != word:
        blob_set_3.add(word)
        

end_time = datetime.now()
print(f"Completed processing at: {end_time.strftime('%H:%M:%S')}")
time_taken(start_time, end_time)

print(len(blob_set_1), len(blob_set_2), len(blob_set_3))

Starting processing at: 00:59:21
Completed processing at: 01:17:36
Time Taken: 0 hours, 18 minutes, 14 seconds
1301 1201 905


Now look at how many each spell checker finds that the other didn't find:

In [56]:
spell_only_1 = spell_set_1 - blob_set_1
spell_only_2 = spell_set_2 - blob_set_2
spell_only_3 = spell_set_3 - blob_set_3

blob_only_1 = blob_set_1 - spell_set_1
blob_only_2 = blob_set_2 - spell_set_2
blob_only_3 = blob_set_3 - spell_set_3

both_3 = spell_set_3 & blob_set_3
either_3 = spell_set_3 | blob_set_3 # this is an OR symbol

print(f"spell / blob 1 have: {len(spell_set_1)} / {len(blob_set_1)}")
print(f"spell / blob 2 have: {len(spell_set_2)} / {len(blob_set_2)}")
print(f"spell / blob 3 have: {len(spell_set_3)} / {len(blob_set_3)}")
print(f"spell unique / blob unique 1 have: {len(spell_only_1)} / {len(blob_only_1)}")
print(f"spell unique / blob unique 2 have: {len(spell_only_2)} / {len(blob_only_2)}")
print(f"spell unique / blob unique 3 have: {len(spell_only_3)} / {len(blob_only_3)}")


spell / blob 1 have: 500 / 1301
spell / blob 2 have: 414 / 1201
spell / blob 3 have: 394 / 905
spell unique / blob unique 1 have: 106 / 907
spell unique / blob unique 2 have: 111 / 898
spell unique / blob unique 3 have: 128 / 639


In [24]:
# display_words_in_context(test_words_1, spell_only_1, False, 3)

In [52]:
blob_oddities = list(blob_set_2 - blob_set_1)
print (' '.join(blob_oddities))
print("\n")
# display_words_in_context(test_words_2, oddities, True, 3)

sleights inwaiting Heinz martialling jacketed tiles pep matching ups newt slated chock longue hog bores sheeted Semitic humouredness mating Karl loverStevens ofhand sheetings vident




In [366]:
len(spell_only_2)

111

In [96]:

print(' '.join(word for word in both_3))
# display_words_in_context(test_words_2, spell_only_2 - spell_only_1, False, 3)

arec simplyaccepting semitic gotto theres granchester thatsall farradays isnt donttake shouldnt celts charleville youve andthe reggie dignitys societys thornley saltash everyones crosby hardacres theyll frau hertfordshire overath muggeridge tavistock itinvolved leant ritz neednt hed helston savour oswald chinaman thatll weve taunton whittaker attachã© loughborough floutings compton manoeuvres istrue whos wasnt moscombe sheetings banterings semitism mursden tosit enamoured wouldnt symonss tj1e youre clementss ifi ribbentrop inwaiting aint seehe reginalds wakefields bremann batmans allhis dorset lto distinguishedand blair lewiss dupont specia1 ohnothing davidson ill1terest dickenson taylors catherines het mosleys villagesir imvery ploughmans carlisles barnets loc saltman lotyou verypleased gentlemans didnt pres redding docrucially ruhr exeter morgans avery hitlers seamus bridewood breadknives eleanor unburdensome friendsand rhineland wilkinson scepticism ribbentrops couldnt anothers anys

In [120]:
errors = "witha bethat selfvident notseek distinguishedand redding civvy ting carryon putout ell arec simplyaccepting gotto thatsall donttake andthe itinvolved istrue tosit tj1e ifi inwaiting seehe allhis lto ohnothing ill1terest het villagesir imvery loc lotyou verypleased docrucially avery friendsand anysuch wer thevery iarrived ican startseeing canonly allshot contemplatin8 infact ot 0f roas agre ofthe dh owardst itsrather ofhand confidentia1 cansee prise loverstevens canbe chainbers lookingback thelikes beyondsuch farradayscircle al1"
errors = errors.split()
errors = sorted(errors)
print (errors)

['0f', 'agre', 'al1', 'allhis', 'allshot', 'andthe', 'anysuch', 'arec', 'avery', 'bethat', 'beyondsuch', 'canbe', 'canonly', 'cansee', 'carryon', 'chainbers', 'civvy', 'confidentia1', 'contemplatin8', 'dh', 'distinguishedand', 'docrucially', 'donttake', 'ell', 'farradayscircle', 'friendsand', 'gotto', 'het', 'iarrived', 'ican', 'ifi', 'ill1terest', 'imvery', 'infact', 'inwaiting', 'istrue', 'itinvolved', 'itsrather', 'loc', 'lookingback', 'lotyou', 'loverstevens', 'lto', 'notseek', 'ofhand', 'ofthe', 'ohnothing', 'ot', 'owardst', 'prise', 'putout', 'redding', 'roas', 'seehe', 'selfvident', 'simplyaccepting', 'startseeing', 'thatsall', 'thelikes', 'thevery', 'ting', 'tj1e', 'tosit', 'verypleased', 'villagesir', 'wer', 'witha']


In [212]:
errors = ['0f', 'agre', 'al1', 'allhis', 'andthe', 'anysuch', 'arec', 'avery', 'be-that', 'beyondsuch', 'canbe', 'canonly', 'cansee', 'carryon', 'Chainbers', 'confidentia1', 'contemplatin8', 'Dh', "'distinguished',and", 'docrucially', "don'ttake", 'T ell', "Farraday'scircle", 'friendsand', 'gotto', 'het', 'Iarrived', 'Ican', "if'-I", 'ill1terest', "I'mvery", 'Infact', 'istrue', 'it,involved', "it'srather", 'loc ked', 'lookingback', 'lot,you', 'lover,Stevens', 'lto', 'not-seek', 'ofthe', 'Oh,nothing', 'ot', 'owardst', 'putout', 'roas ting', 'seehe', 'self-vident', 'simplyaccepting', 'startseeing', "that'sall", 'thelikes', 'thevery', 'tJ1e', 'tosit', 'verypleased', 'village,sir', 'wer', 'with-a']


In [215]:
corrections = ['of', 'agree', 'all', 'all his', 'and the', 'any such', 'care', 'a very', 'be that', 'beyond such', 'can be', 'can only', 'can see', 'carry on', 'Chambers', 'confidential', 'contemplating', 'Oh', "'distinguished', and", 'do crucially', "don't take", 'Tell', 'Farradays Circle', 'friends and', 'got to', 'the', 'I arrived', 'I can', 'if I', 'interest', "I'm very", 'In fact', 'is true', 'it involved', "its rather", 'locked', 'looking back', 'lot, you', 'lover, Stevens', 'to', 'not seek', 'of the', 'Oh, nothing', 'to', 'towards', 'put out', 'roasting', 'see he', 'self-evident', 'simply accepting', 'start seeing', "that's all", 'the likes', 'the very', 'the', 'to sit', 'very pleased', 'village, sir', 'were', 'with a']

In [216]:
for error, correction in zip(errors, corrections):
    if error == correction:
        display_words_in_context(test_words_3, [error], False, 20)

In [217]:
for word in errors:
    if word not in text:
        display_words_in_context(test_words_3, [word], False, 6)

In [199]:
display_text_in_context(text, ["never forgive me"])

tly, sir, I don't know what else you could do at this hour. The wife would never forgive me if'-I were to let you away into the night."  Thus it was th


In [98]:
print([word in blob_set_3 for word in errors])

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [210]:
where_is("ell")

In blob set 1
certain matter weve been debating T [ell] me do you suppose the debt
In blob  set 2
certain matter weve been debating T [ell] me do you suppose the debt
In blob set 3
certain matter weve been debating t [ell] me do you suppose the debt


In [209]:
display_text_in_context(text,["ell"])

really think you should take a break. I'll foot the bill for the gas. You fellows, you're always locked up in these big Houses helping out, how do you 


In [218]:
replacements = {error: correction for error, correction in zip (errors, corrections)}

In [221]:
print(replacements["with-a"])

with a


In [374]:
print(' '.join(word for word in (blob_only_2 - blob_only_1)))
display_words_in_context(test_words_2, blob_only_2 - blob_only_1, False, 3)

ups newt tiles sheeted sleights pep mating slated hog jacketed bores chock matching
and the turn [ups] of my trousers
as falconing or [newt] mating attributes none
other than roof [tiles] and guttering The
would be dust [sheeted] leaving all the
as the various [sleights] ofhand the equivalent
a military style [pep] talk impressing upon
falconing or newt [mating] attributes none of
clusters of dark [slated] roofs here and
understand So much [hog] wash has been
many stern dark [jacketed] gentlemen sometimes sitting
permit the twelve [bores] to be used
his foot is [chock] full of notes
filled with ill [matching] armchairs and occasional


In [376]:
print(' '.join(word for word in (spell_only_3 - spell_only_2 - spell_only_1)))
display_words_in_context(test_words_3, spell_only_3 - spell_only_2 - spell_only_1, False, 3)

martha boston austin alice darlington morgan thursday james leslie boer mr italians mortimer englishmen campbell georges andrews southampton stevens london david meredith europe williams frenchmen berkshire william halifax british swiss scotland hayes jewish dr mrs christmas iii britannica roosevelt paris astor american frenchman germans sydney sunday versailles englishman johnson taylor belgians simpson herr dorothy eden agnes jones sarah britain ok lisa lloyd karl catherine winston yorkshire tory churchill italian cornwall pennsylvania oxfordshire americans jews richard september england belgian charles april edward dave george
to speak to [martha] or dorothy or
mr farraday amidst [boston] society they paid
formidable mrs eleanor [austin] at that time
in mind of [alice] white do you
me away from [darlington] hall for as
to me as [morgan] sir trevor morgan
off again on [thursday] miss kenton on
butler to sir [james] chambers and mr
wireless his names [leslie] mandrake i just
attack on

In [377]:
print(' '.join(word for word in (blob_only_3 - blob_only_2 - blob_only_1)))
display_words_in_context(test_words_3, blob_only_3 - blob_only_2 - blob_only_1, False, 3)

viscount continentals silvers mandrake bolsheviks hmm fascists nil bolshevik spencer cardinals grahams rosemary
the portrait of [viscount] wetherby when my
this is true [continentals] are unable to
of mr john [silvers] the industrialist where
his names leslie [mandrake] i just wondered
french and the [bolsheviks] im very sorry
a comic figure [hmm] thats taking it
british union of [fascists] such claims can
to recognize the [nil] significance of silver
the proprieties of [bolshevik] russia this provoked
you stevens mr [spencer] here wishes a
way from mr [cardinals] writing desk gave
objection to mr [grahams] analogy was the
pleased to hire [rosemary] and agnes on


106
111
128
907
898
639


In [256]:
counter = 0
for word in blob_set_2:
    if word.lower() in blob_set_3:
        print(f"{word} in both")
        print(Word(word).spellcheck())
        if word != word.lower():
            print(Word(word.lower()).spellcheck())
    else:
        print (f"{word} in 2 but not in 3")
        print(Word(word).spellcheck())
    print("\n")    
    counter = counter + 1
    if counter > 15: break    

Barnets in both
[('Markets', 0.31543624161073824), ('Parents', 0.28859060402684567), ('Harness', 0.18120805369127516), ('Carpets', 0.087248322147651), ('Earnest', 0.06711409395973154), ('Earners', 0.053691275167785234), ('Hornets', 0.006711409395973154)]
[('markets', 0.2554347826086957), ('parents', 0.23369565217391305), ('harness', 0.14673913043478262), ('bayonets', 0.07608695652173914), ('carpets', 0.07065217391304347), ('earnest', 0.05434782608695652), ('earners', 0.043478260869565216), ('banners', 0.03260869565217391), ('barrels', 0.02717391304347826), ('barges', 0.010869565217391304), ('barest', 0.010869565217391304), ('hornets', 0.005434782608695652), ('baskets', 0.005434782608695652), ('baronet', 0.005434782608695652), ('baroness', 0.005434782608695652), ('barns', 0.005434782608695652), ('bargees', 0.005434782608695652), ('badness', 0.005434782608695652)]


TONIGHT in 2 but not in 3
[('TONIGHT', 0.0)]


Carlisle in both
[('Carlisle', 0.0)]
[('carlyle', 1.0)]


Sit in 2 but not i

In [ ]:
errors = "istrue canbe im tj1e overath laval itsrather wer silverss gotto thelikes ifi 0f prise ritz ofthe andthe lto lookingback al1 verypleased het thevery agre bethat seehe ot selfvident sleightsofhand cansee thatsall distinguishedand villagesir docrucially arec farradayscircle witha canonly barnets redding tosit clementss newtmating ill1terest anysuch dh lewiss donttake allhis contemplatin8 startseeing symons owardst imvery simplyaccepting selftraining wellcontented evercourteous donttake allhis lastminute civvy"

In [181]:
bad_words=["ohnothing", "iarrived", "friendsand", "allshot"]
for word in bad_words:
    print(word in errors_split)

False
False
False
False


In [186]:
suggestions_split = suggestions.split(' ')
suggestions_sorted = sorted(suggestions_split)
print(suggestions_sorted)

['1e', "Barnet's", "Clements's", "Farraday'scircle", "I'mvery", 'Laval', "Lewis's", 'Overath', 'Redding', 'Ritz', "Silvers's", 'Symons', 'be-that', 'd-h', "don'ttake", "don'ttake", 'ever-courteous', "it'srather", 'last-minute', 'newt-mating', 'self-training', 'self-vident', 'sleights-of-hand', "that'sall", 'well-contented', 'with-a']


In [188]:
errors_sorted = sorted(errors_split)
print (errors_sorted)

['0f', 'agre', 'al1', 'allhis', 'allhis', 'allshot', 'andthe', 'anysuch', 'arec', 'barnets', 'bethat', 'canbe', 'canonly', 'cansee', 'civvy', 'clementss', 'contemplatin8', 'dh', 'distinguishedand', 'docrucially', 'donttake', 'donttake', 'evercourteous', 'farradayscircle', 'friendsand', 'gotto', 'het', 'iarrived', 'ifi', 'ill1terest', 'im', 'imvery', 'istrue', 'itsrather', 'lastminute', 'laval', 'lewiss', 'lookingback', 'lto', 'newtmating', 'ofthe', 'ohnothing', 'ot', 'overath', 'owardst', 'prise', 'redding', 'ritz', 'seehe', 'selftraining', 'selfvident', 'silverss', 'simplyaccepting', 'sleightsofhand', 'startseeing', 'symons', 'thatsall', 'thelikes', 'thevery', 'tj1e', 'tosit', 'verypleased', 'villagesir', 'wellcontented', 'wer', 'witha']


In [184]:
for word in bad_words:
    errors_split.append(word)
print(errors_split)

['istrue', 'canbe', 'im', 'tj1e', 'overath', 'laval', 'itsrather', 'wer', 'silverss', 'gotto', 'thelikes', 'ifi', '0f', 'prise', 'ritz', 'ofthe', 'andthe', 'lto', 'lookingback', 'al1', 'verypleased', 'het', 'thevery', 'agre', 'bethat', 'seehe', 'ot', 'selfvident', 'sleightsofhand', 'cansee', 'thatsall', 'distinguishedand', 'villagesir', 'docrucially', 'arec', 'farradayscircle', 'witha', 'canonly', 'barnets', 'redding', 'tosit', 'clementss', 'newtmating', 'ill1terest', 'anysuch', 'dh', 'lewiss', 'donttake', 'allhis', 'contemplatin8', 'startseeing', 'symons', 'owardst', 'imvery', 'simplyaccepting', 'selftraining', 'wellcontented', 'evercourteous', 'donttake', 'allhis', 'lastminute', 'civvy', 'ohnothing', 'iarrived', 'friendsand', 'allshot']


In [185]:
print(suggestions)

Overath Laval 1e with-a it'srather Silvers's Farraday'scircle Barnet's Clements's newt-mating Lewis's don'ttake I'mvery self-training well-contented ever-courteous don'ttake last-minute Ritz Redding self-vident sleights-of-hand be-that that'sall d-h Symons


In [ ]:
errors_split = ['istrue', 'canbe', 'im', 'tj1e', 'overath', 'laval', 'itsrather', 'wer', 'silverss', 'gotto', 'thelikes', 'ifi', '0f', 'prise', 'ritz', 'ofthe', 'andthe', 'lto', 'lookingback', 'al1', 'verypleased', 'het', 'thevery', 'agre', 'bethat', 'seehe', 'ot', 'selfvident', 'sleightsofhand', 'cansee', 'thatsall', 'distinguishedand', 'villagesir', 'docrucially', 'arec', 'farradayscircle', 'witha', 'canonly', 'barnets', 'redding', 'tosit', 'clementss', 'newtmating', 'ill1terest', 'anysuch', 'dh', 'lewiss', 'donttake', 'allhis', 'contemplatin8', 'startseeing', 'symons', 'owardst', 'imvery', 'simplyaccepting', 'selftraining', 'wellcontented', 'evercourteous', 'donttake', 'allhis', 'lastminute', 'civvy', 'ohnothing', 'iarrived', 'friendsand', 'allshot']

In [51]:


for bad_word in bad_words:
    ind = text.find(bad_word)
    if ind == -1:
        print(f"{bad_word} not found")
    else:
        window = text[ind-50:ind+50]
        window = window.replace("\n", " ")
        window = window.replace("\t", " ")
        print(window)
    #test_result = (Word(test_word).spellcheck())
    #print(test_result)


ohnothing not found
iarrived not found
to."  "You don't understand, Stevens. Well, we're friendsand so I'll put it to you frankly. Over the
allshot not found


In [71]:
test_words = text.translate(str.maketrans('', '', string.punctuation)).lower().split()
bad_words=["ohnothing", "iarrived", "friendsand", "allshot"]
bad_words = ["ell", "ting"]
for bad_word in bad_words:
    try:
        ind = test_words.index(bad_word)
        print(f"The index of '{bad_word}' is: {ind}")
        print(' '.join(test_words[ind - 10:ind + 10]))
    except ValueError:
        print(f"'{item_to_find}' not found in the list.")


The index of 'ell' is: 61381
your help on a certain matter weve been debating t ell me do you suppose the debt situation regarding america
The index of 'ting' is: 34786
apron the room was dominated by the smell of roas ting dr meredith rose and said my condolences stevens he


In [58]:
print(' '.join(blob_only))

farsighted undeniable rapping celebratory topnotch contexts vouch residency overawed breakages professionalized vowel illuminates bolting impractical nostalgic judgements replenishing snobbish unprepared poplar remotely teapots equanimity unforgettable nuance fulcrum positioned aforementioned idealism ripple scholarly whats hogwash chump flimsy signpost pandemonium unburden starkness dependable duplicitous surreptitious finery tablecloth pleasantries bereavement thawed misspelt wallpaper eased leaven diners advertises indisputable wended awesome gent chillier endlessly daunting imaginable tensions hmm camaraderie indelible nonsensical fracas stoked drunks seeping bungling throwaway humbling overtired disruptive generational platter grandchild sightseeing pessimistic hedgerows endearing complicit mans vendetta fluke largeness confusions demonstrativeness staffing breakfasted carryon infuriatingly pecking pastries outmoded relented woken alarmist focusing encyclopedias frogmarched silver

In [64]:
both = spell_set & blob_set
print(len(both))

270


In [65]:
print(' '.join(word for word in both))

valetbutler twentyeight wellregarded midmorning chalmers wellwritten istrue benn tj1e wellbuilt theyve muggeridge manoeuvred ploughmans bridewood hardbacked inflexions blair bridgeposts canbe thats youre midconversation ribbentrop oakpanelled oswald shant infact unrevealing farradays fortynine exeter itsrather maynard manoeuvring compton overath wer theres dinnerwaiting laval avery underservants salisbury peptalks beyondsuch brigoon jewry itll militarystyle forwardlooking bentley reggie thirtythree longdistance hardacres ruhr davids rayne middleaged silverss dustsheeted silverpolishing everpresent blackshirts humphreys bremann hasnt saltman halflight aint attachã© dustsheetings faroff notseek wakefield firstclass trevor im savour helston fulltime twentythree womans thirtyfive latters selfevident davidson weve gotto mosley highflown manservants darkslated chinamen hitlers thelikes streethawkers 0f ifi thornton goodsized wetherby keynes civvy hed taylors carolyn kenton whitehaired selfre

In [99]:
original_errors = "istrue canbe im tj1e overath laval itsrather wer silverss gotto thelikes ifi 0f prise ritz ofthe andthe lto lookingback al1 verypleased het thevery agre bethat seehe ot selfvident sleightsofhand cansee thatsall distinguishedand villagesir docrucially arec farradayscircle witha canonly barnets redding tosit clementss newtmating ill1terest anysuch dh lewiss donttake allhis contemplatin8 startseeing symons owardst imvery simplyaccepting selftraining wellcontented evercourteous donttake allhis lastminute civvy"

In [102]:
original_errors_split = set(original_errors.split())

In [105]:
missed = {word for word in original_errors_split if word not in errors}

In [108]:
print ([(word, word in blob_set_3) for word in missed])

[('clementss', True), ('laval', True), ('silverss', True), ('wellcontented', False), ('newtmating', False), ('distinguishedand', True), ('lewiss', True), ('bethat', False), ('overath', True), ('symons', True), ('ritz', True), ('witha', False), ('lastminute', False), ('barnets', True), ('im', True), ('selftraining', False), ('redding', True), ('sleightsofhand', False), ('selfvident', False), ('evercourteous', False), ('civvy', True)]


In [109]:
print ([(word, word in both_3) for word in missed])

[('clementss', True), ('laval', True), ('silverss', True), ('wellcontented', False), ('newtmating', False), ('distinguishedand', True), ('lewiss', True), ('bethat', False), ('overath', True), ('symons', True), ('ritz', True), ('witha', False), ('lastminute', False), ('barnets', True), ('im', True), ('selftraining', False), ('redding', True), ('sleightsofhand', False), ('selfvident', False), ('evercourteous', False), ('civvy', True)]


In [111]:
print ([(word, word in spell_set_3) for word in missed])


[('clementss', True), ('laval', True), ('silverss', True), ('wellcontented', False), ('newtmating', False), ('distinguishedand', True), ('lewiss', True), ('bethat', False), ('overath', True), ('symons', True), ('ritz', True), ('witha', False), ('lastminute', False), ('barnets', True), ('im', True), ('selftraining', False), ('redding', True), ('sleightsofhand', False), ('selfvident', False), ('evercourteous', False), ('civvy', True)]


In [178]:
for word in errors_split:
    if word not in text:
        if word not in cleaned_suggestions:
            print(word)


tj1e
distinguishedand
villagesir


In [117]:
where_is("distinguishedand")

In spell set 1
In blob set 1
businessmen or the newly rich as [distinguishedand] in my opinion this piece of
In spell set 2
In blob  set 2
businessmen or the newly rich as [distinguishedand] in my opinion this piece of
In spell set 3
In blob set 3
businessmen or the newly rich as [distinguishedand] in my opinion this piece of


In [110]:
print([word for word in blob_set_3 if word not in blob_only])

NameError: name 'blob_only' is not defined

In [173]:
suggestions = "Overath Laval 1e with-a it'srather Silvers's Farraday'scircle Barnet's Clements's newt-mating Lewis's don'ttake I'mvery self-training well-contented ever-courteous don'ttake last-minute Ritz Redding self-vident sleights-of-hand be-that that'sall d-h Symons"
cleaned_suggestions = suggestions.translate(str.maketrans('', '', string.punctuation)).lower().split()

In [174]:
for word in suggestions.split(' '):
    if word not in text:
        print(word)

In [117]:
for word in errors_split:
    if word not in cleaned_suggestions:`
        if word not in test_words:
            print(word)

In [179]:
" prise " in text

True